# 3.4 Seguridad Básica en SQL Server

**Resumen Ejecutivo**: Este módulo introduce los principios fundamentales de seguridad en bases de datos relacionales, con énfasis en el modelo de **privilegios mínimos** y la gestión de **roles de usuario**. La seguridad no es un "add-on" opcional: es un pilar crítico para la protección de información sensible, el cumplimiento regulatorio (GDPR, HIPAA, SOX) y la prevención de accesos no autorizados que pueden resultar en pérdidas financieras o daño reputacional.

---

## 📚 Objetivos de Aprendizaje

Al finalizar este módulo serás capaz de:

1. **Comprender** la diferencia entre autenticación (identificación) y autorización (permisos)
2. **Aplicar** el principio de privilegios mínimos para minimizar superficie de ataque
3. **Crear** usuarios y asignar permisos granulares según necesidades del negocio
4. **Diseñar** roles personalizados para diferentes perfiles de usuario (analista, desarrollador, administrador)
5. **Auditar** permisos existentes e identificar riesgos de seguridad

---

## 🔐 Prerequisitos

Antes de comenzar, debes estar familiarizado con:

- **Conceptos de SQL Server**: instancias, bases de datos, esquemas
- **Comandos DDL básicos**: `CREATE`, `ALTER`, `DROP`
- **Nociones de arquitectura empresarial**: separación de ambientes (dev, test, prod)
- **Conocimiento del modelo de objetos**: tablas, vistas, procedimientos almacenados

---

## 🔑 Conceptos Fundamentales de Seguridad

### 🔵 Autenticación vs Autorización

| Concepto | Pregunta | Ejemplo |
|----------|----------|---------|
| **Autenticación** | *¿Quién eres?* | Login con credenciales (`CREATE LOGIN`) |
| **Autorización** | *¿Qué puedes hacer?* | Permisos sobre objetos (`GRANT SELECT`) |

### 🔵 Jerarquía de Seguridad en SQL Server

```
Instancia SQL Server
├── Logins (nivel servidor)
│   └── Server Roles (sysadmin, dbcreator, etc.)
└── Bases de Datos
    ├── Usuarios (mapeados a logins)
    │   └── Database Roles (db_owner, db_datareader, etc.)
    └── Permisos directos sobre objetos
```

### 🔵 Principio de Privilegios Mínimos

> **Regla de Oro**: Otorgar **solo** los permisos estrictamente necesarios para completar las tareas asignadas.

**Consecuencias de violar este principio**:
- ❌ Superficie de ataque ampliada (mayor vulnerabilidad ante SQL injection)
- ❌ Riesgo de eliminación accidental de datos críticos
- ❌ Dificultad para rastrear responsabilidades en auditorías
- ❌ Incumplimiento de normativas de protección de datos

---

## 📊 Caso de Uso Práctico: Usuario Analista de Reportes

### 🎯 Escenario Real

Tu empresa contrata a un analista de BI que necesita generar reportes de ventas trimestrales. Este usuario debe:

✅ **Leer** datos de tablas `dim_clientes`, `dim_productos`, `fact_ventas`  
✅ **Ejecutar** vistas preexistentes (`vw_ventas_mensuales`)  
❌ **NO modificar** datos (sin `INSERT`, `UPDATE`, `DELETE`)  
❌ **NO crear** objetos nuevos (sin `CREATE TABLE`, `ALTER SCHEMA`)

### 🛠️ Implementación Step-by-Step

#### **Paso 1: Crear el Login (Nivel Servidor)**

Primero creamos la identidad en el servidor SQL Server. En entornos corporativos se prefiere autenticación de Windows (Active Directory), pero para laboratorios usamos SQL Authentication:

In [ ]:
-- Crear login con política de contraseñas robustas
CREATE LOGIN analista_reportes 
WITH PASSWORD = 'P@ssw0rd#2024!',
     CHECK_POLICY = ON,        -- Obliga a cumplir política de complejidad
     CHECK_EXPIRATION = ON;    -- Contraseña expira según configuración del servidor

-- Nota: En producción, usar CHECK_EXPIRATION = ON es crítico para seguridad

#### **Paso 2: Mapear Login a Usuario en la Base de Datos**

El login solo identifica al usuario en el servidor. Para acceder a una base de datos específica, necesitamos crear un usuario mapeado:

In [ ]:
USE DataWarehouse;  -- Cambiar al contexto de la base de datos
GO

CREATE USER analista_reportes 
FOR LOGIN analista_reportes;

-- En este punto el usuario existe pero NO tiene permisos aún (acceso bloqueado por defecto)

#### **Paso 3: Asignar Rol de Solo Lectura**

SQL Server incluye roles predefinidos. Para lectura usamos `db_datareader` (acceso `SELECT` a todas las tablas):

In [ ]:
-- Asignar rol de solo lectura
ALTER ROLE db_datareader ADD MEMBER analista_reportes;

-- Verificar permisos asignados
SELECT 
    USER_NAME(member_principal_id) AS Usuario,
    USER_NAME(role_principal_id) AS Rol
FROM sys.database_role_members
WHERE USER_NAME(member_principal_id) = 'analista_reportes';

#### **Paso 4 (Opcional): Permisos Granulares con Rol Personalizado**

Si `db_datareader` otorga demasiados permisos (acceso a **todas** las tablas), creamos un rol personalizado:

In [ ]:
-- Crear rol personalizado para reportes de ventas
CREATE ROLE rol_reportes_ventas;

-- Otorgar permisos SELECT solo en tablas específicas
GRANT SELECT ON dbo.dim_clientes TO rol_reportes_ventas;
GRANT SELECT ON dbo.dim_productos TO rol_reportes_ventas;
GRANT SELECT ON dbo.fact_ventas TO rol_reportes_ventas;

-- Asignar usuario al rol personalizado
ALTER ROLE rol_reportes_ventas ADD MEMBER analista_reportes;

-- Ventaja: Control granular. Si se agregan nuevas tablas sensibles, NO se accede automáticamente.

#### **Paso 5: Auditoría de Permisos Efectivos**

Después de configurar seguridad, **siempre** verificar permisos reales del usuario:

In [ ]:
-- Simular contexto de ejecución del usuario (requiere permisos de administrador)
EXECUTE AS USER = 'analista_reportes';

-- Listar permisos efectivos del usuario actual
SELECT 
    OBJECT_SCHEMA_NAME(major_id) AS Esquema,
    OBJECT_NAME(major_id) AS Objeto,
    permission_name AS Permiso,
    state_desc AS Estado
FROM sys.database_permissions
WHERE grantee_principal_id = USER_ID();

-- Volver al contexto administrativo
REVERT;

---

## 🎯 Ejercicios Guiados

### 🟢 Ejercicio 1: Permisos Mínimos para Usuario Analista

**Objetivo**: Identificar los permisos estrictamente necesarios para un usuario que solo genera reportes.

**Instrucciones**:
1. Crear un login `usuario_lectura` con contraseña robusta
2. Mapear el login a un usuario en la base de datos `DataWarehouse`
3. Otorgar **solo** permisos `SELECT` en las tablas `dim_clientes`, `fact_ventas`
4. Verificar que NO puede ejecutar `INSERT`, `UPDATE` ni `DELETE`

**Criterio de éxito**: El usuario puede ejecutar `SELECT * FROM fact_ventas` pero recibe error al intentar `DELETE FROM fact_ventas`.

---

### 🟠 Ejercicio 2: Diseño de Roles Granulares

**Objetivo**: Crear 3 roles con diferentes niveles de acceso.

**Roles a implementar**:

| Rol | Permisos | Usuarios Típicos |
|-----|----------|------------------|
| `rol_lectura` | `SELECT` en todas las tablas | Analistas, auditores |
| `rol_escritura` | `SELECT`, `INSERT`, `UPDATE` (sin `DELETE`) | Aplicaciones de ingesta |
| `rol_admin` | Control total (`db_owner`) | DBAs, desarrolladores senior |

**Instrucciones**:
1. Crear los 3 roles con `CREATE ROLE`
2. Asignar permisos usando `GRANT`
3. Crear 3 usuarios de prueba y asignarlos a roles diferentes
4. Probar con `EXECUTE AS USER` que cada rol tiene permisos correctos

---

### 🔴 Reto: Auditoría Trimestral de Permisos

**Objetivo**: Diseñar un proceso de auditoría de seguridad que identifique permisos no utilizados.

**Requerimientos**:
1. Consultar vistas del sistema `sys.database_permissions`, `sys.database_role_members`
2. Cruzar con logs de actividad (tabla de auditoría o Extended Events)
3. Identificar usuarios que NO han ejecutado consultas en 90 días
4. Generar recomendación de permisos a revocar

**Entregable**: Script SQL que retorne lista de usuarios candidatos a revisión con justificación.

---

## 📈 Perspectiva de Negocio: ROI de la Seguridad

### 💰 Impacto Financiero de Brechas de Seguridad

**Costos promedio por incidente (industria 2024)**:

| Tipo de Incidente | Costo Promedio | Tiempo Recuperación |
|-------------------|----------------|---------------------|
| Filtración de datos (PII) | USD $4.45M | 287 días |
| Ransomware | USD $1.85M | 49 días |
| Acceso no autorizado (insider) | USD $648K | 91 días |

*Fuente: IBM Cost of Data Breach Report 2024*

### 📊 Costo/Beneficio de Implementar Seguridad Robusta

**Inversión inicial** (empresa mediana 50-200 empleados):
- **Consultoría de seguridad**: USD $15K-$30K (diseño de arquitectura de permisos)
- **Herramientas de auditoría**: USD $5K-$10K/año (Extended Events, alertas automáticas)
- **Capacitación de equipo**: USD $3K-$5K (certificaciones en SQL Server Security)
- **TOTAL**: USD $23K-$45K primer año

**Beneficios tangibles**:
- **Prevención de multas regulatorias**: GDPR hasta €20M (4% facturación anual)
- **Reducción de riesgo de fraude**: 67% menor probabilidad de incidentes internos
- **Auditorías más rápidas**: 40% reducción en tiempo de auditoría SOX/HIPAA
- **Seguro cibernético más económico**: 25% descuento con controles certificados

### 🎯 ROI Proyectado (5 años)

Para empresa mediana con base de datos de clientes (500K registros):

| Año | Inversión | Incidentes Evitados | Ahorro Estimado | ROI Acumulado |
|-----|-----------|---------------------|-----------------|---------------|
| 1 | -$40K | 1 breach menor | +$150K | **+275%** |
| 2 | -$10K | 2 intentos de acceso no autorizado | +$80K | **+625%** |
| 3-5 | -$10K/año | Multas GDPR evitadas | +$200K/año | **+1,800%** |

**Conclusión**: Cada dólar invertido en seguridad proactiva retorna $18 en 5 años (vs costos reactivos de brechas).

---

## 💼 Aplicación Práctica en el Mundo Real

### 🏥 Caso: Sistema Hospitalario (HIPAA Compliance)

**Contexto**: Un hospital necesita proteger historiales médicos electrónicos (EHR) bajo regulación HIPAA.

**Implementación de seguridad**:

| Usuario | Rol | Permisos | Justificación |
|---------|-----|----------|---------------|
| Médicos | `rol_medicos` | `SELECT`, `UPDATE` en `pacientes`, `diagnosticos` | Necesitan leer y actualizar historiales |
| Enfermeras | `rol_enfermeria` | `SELECT` en `pacientes`, `INSERT` en `signos_vitales` | Solo registran mediciones |
| Facturación | `rol_facturacion` | `SELECT` en `pacientes.id`, `pacientes.nombre` (sin datos clínicos) | Solo necesitan datos demográficos |
| Auditoría | `rol_auditoria` | `SELECT` en todas las tablas (solo lectura) | Investigaciones de incidentes |

**Métricas de cumplimiento**:
- ✅ 100% usuarios con credenciales individuales (no compartidas)
- ✅ Auditoría habilitada (log de cada acceso a datos sensibles)
- ✅ Segregación de funciones (médicos NO pueden modificar facturación)

---

### 🏦 Caso: Plataforma Financiera (SOX Compliance)

**Contexto**: Banco necesita prevenir fraude interno y cumplir Sarbanes-Oxley (SOX).

**Arquitectura de seguridad**:

```
Producción (solo lectura)
├── Aplicación Web → usuario_app (SELECT en vistas, EXECUTE en SPs)
└── Analistas BI → usuario_reporting (SELECT en tablas agregadas)

Desarrollo/Test
├── Desarrolladores → db_owner (control total en ambiente no productivo)
└── QA → db_datareader + db_datawriter (pruebas de carga)
```

**Controles SOX implementados**:
1. **Segregación de ambientes**: Desarrolladores SIN acceso a producción
2. **Cambios auditados**: Todo cambio en producción requiere ticket aprobado
3. **Revisión de accesos**: Trimestral, eliminación de cuentas inactivas
4. **Principio de 4 ojos**: Cambios de permisos requieren aprobación dual

**Resultado**: Auditorías SOX pasadas con 0 observaciones en últimos 3 años.

---

## 📌 Conclusiones Clave

### 🎓 Conceptos Fundamentales

1. **Seguridad en capas**: Defensa en profundidad (firewall + autenticación + permisos + cifrado)
2. **Principio de mínimos privilegios**: Base para reducir superficie de ataque
3. **Roles > Permisos directos**: Escalabilidad y mantenibilidad a largo plazo
4. **Auditoría continua**: La seguridad no es "set and forget", requiere revisión periódica

### 🔑 Aprendizajes Críticos

- **Autenticación** responde "¿Quién?" → **Autorización** responde "¿Qué puede hacer?"
- **Login** (nivel servidor) ≠ **Usuario** (nivel base de datos) → Mapeo obligatorio
- **Roles predefinidos** (`db_datareader`) son punto de partida, roles personalizados para control fino
- **`EXECUTE AS USER`** es herramienta esencial para pruebas de seguridad

### 🚀 Siguientes Pasos

Tras dominar seguridad básica, los siguientes temas lógicos son:

1. **Seguridad avanzada**: Row-Level Security (RLS), Dynamic Data Masking, Transparent Data Encryption (TDE)
2. **Auditoría con Extended Events**: Captura de todas las operaciones de modificación de datos
3. **Gestión de secretos**: Azure Key Vault, Credential Manager para conexiones seguras

---

## ⚠️ Buenas Prácticas y Errores Comunes

### ✅ Mejores Prácticas

| Práctica | Justificación |
|----------|---------------|
| **Usar roles en lugar de permisos directos** | Facilita mantenimiento (cambias el rol, no cada usuario) |
| **Autenticación de Windows en producción** | Integración con Active Directory, políticas centralizadas |
| **Habilitar auditoría de login** | Detecta intentos de acceso no autorizado |
| **Rotar contraseñas regularmente** | Mitigar riesgo de credenciales comprometidas |
| **Esquemas segregados por función** | `ventas.fact_ventas` vs `finanzas.transacciones` (aislamiento lógico) |
| **Revisión trimestral de permisos** | Eliminar "role creep" (acumulación de privilegios) |

### ❌ Errores Críticos a Evitar

| Error | Impacto | Remediación |
|-------|---------|-------------|
| **Otorgar `db_owner` por defecto** | Usuario puede eliminar base completa | Usar roles granulares (`db_datareader`, `db_datawriter`) |
| **Usar contraseñas débiles** | Vulnerable a ataques de fuerza bruta | `CHECK_POLICY = ON`, mínimo 12 caracteres |
| **No documentar permisos** | Imposible auditar en caso de incidente | Mantener tabla de asignación de roles |
| **Permisos en cuenta de servicio compartida** | No se puede rastrear quién ejecutó qué | Usuario individual por persona, cuenta servicio solo para aplicaciones |
| **Olvidar revocar permisos al cambio de rol** | Ex-empleados con acceso activo | Proceso de offboarding automatizado |

### 🔒 Checklist de Seguridad Pre-Producción

Antes de promover cambios a producción, verificar:

- [ ] **Todos** los usuarios tienen contraseña con `CHECK_POLICY = ON`
- [ ] **Ningún** usuario tiene `sysadmin` o `db_owner` sin justificación
- [ ] **Todas** las cuentas de aplicación usan Windows Authentication
- [ ] **Existe** bitácora de cambios de permisos (tabla de auditoría)
- [ ] **Se probó** que usuarios NO pueden acceder a datos sensibles fuera de su alcance

---

## 📚 Recursos Complementarios

### 📖 Documentación Oficial

- [SQL Server Security Best Practices](https://learn.microsoft.com/sql/relational-databases/security/security-center-for-sql-server-database-engine) - Microsoft Learn
- [Database-Level Roles](https://learn.microsoft.com/sql/relational-databases/security/authentication-access/database-level-roles) - Roles predefinidos y personalizados
- [Permissions Hierarchy](https://learn.microsoft.com/sql/relational-databases/security/permissions-database-engine) - Modelo de herencia de permisos

### 🎥 Tutoriales Recomendados

- *SQL Server Security Fundamentals* (Pluralsight) - 4.5 horas
- *Database Security for Developers* (LinkedIn Learning) - 2 horas

### 🏆 Certificaciones Profesionales

- **Microsoft Certified: Azure Database Administrator Associate** - Incluye módulo de seguridad avanzada
- **Certified Information Systems Security Professional (CISSP)** - Dominio 8: Software Development Security

---

## 🔖 Pie Editorial

**Curso**: Fundamentos de SQL Server - Nivel Intermedio  
**Módulo**: 3.4 Seguridad Básica en Bases de Datos  
**Versión**: 2.0 (Actualizado Enero 2025)  
**Autor**: lraigosov / LuisRai  
**Licencia**: Uso educativo - Atribución requerida

> 💡 **Nota sobre IA**: Este material fue estructurado y enriquecido con asistencia de modelos de lenguaje (OpenAI GPT-4, Anthropic Claude), pero todo el contenido técnico fue validado, las decisiones pedagógicas fueron tomadas y las aplicaciones de negocio fueron diseñadas por humanos con experiencia en administración de bases de datos empresariales.

---
## Navegación
[⬅️ Anterior](03_particiones_basico.ipynb) | [Siguiente ➡️](05_transacciones_bloqueos.ipynb)
---
